<a href="https://colab.research.google.com/github/CaoHaiNam/Nam3/blob/master/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam, RMSprop
import numpy as np
import random
from tqdm import tqdm_notebook

(X_train, y_train), (X_test, y_test) = mnist.load_data()


In [0]:
# pre-processing
X_train = X_train.reshape(60000, 784)
X_test  = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')/255
X_test  = X_test.astype('float32')/255

In [0]:
# dim of noise vector
z_dim = 100

In [0]:
#optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

In [0]:
# Generator model
g = Sequential()
g.add(Dense(256, input_dim = z_dim, activation = LeakyReLU(alpha=0.2)))
g.add(Dense(512, activation = LeakyReLU(alpha=0.2)))
g.add(Dense(1024, activation = LeakyReLU(alpha=0.2)))
# sigmoid, because data was nolmalized
g.add(Dense(784, activation='sigmoid'))
g.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
g.summary()

In [0]:
# Discriminator model
d = Sequential()
d.add(Dense(1024, input_dim = 784, activation = LeakyReLU(alpha=0.2)))
d.add(Dropout(0.3))
d.add(Dense(512, activation = LeakyReLU(alpha=0.2)))
d.add(Dropout(0.3))
d.add(Dense(256, activation = LeakyReLU(0.2)))
d.add(Dropout(0.3))
d.add(Dense(1, activation = 'sigmoid'))
d.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
d.summary()

In [0]:
d.trainable = False
inputs = Input(shape=(z_dim, ))
hidden = g(inputs)
output = d(hidden)
gan = Model(inputs, output)
gan.compile(loss='binary_crossentropy', optimizer=adam, mareics = ['accuracy'])
gan.summary()

In [0]:
# draw loss function
def plot_loss(losses):
  d_loss = [v[0] for v in losses["D"]]
  g_loss = [v[0] for v in losses["G"]]

  plt.figure(figsize = (10,8))
  plt.plot(d_loss, label='Discriminator loss')
  plt.plot(g_loss, label='Generator loss')

  plt.xlabel('Epoches')
  plt.ylabel('Loss')
  plt.legend()
  plt.show()

In [0]:
# function drawing sample from Generator
def plot_generated(n_ex=10, dim=(1, 10), figsize=(12, 2)):
  noise = np.random.normal(0, 1, zise = (n_ex, z_dim))
  generated_images = g.predict(noise)
  generated_images = generated_images.reshape(n_ex, 28, 28)

  plt.figure(figsize = figsize)
  for i in range(generated_images.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generated_images[i], interpolation = 'nearest', cmap = 'gray')
    plt.axis('off')
  plt.tight_layout()
  plt.show()

In [0]:
# store loss and accuracy of D and G
losses = {"D":[], "G":[]}

def train(epoches=1, plt_frq=1, BATCH_SIZE=128):
  #the amount of running times per epoch
  batchCount = int(X_train.shape[0]/ BATCH_SIZE)
  print('Epochs: ', epochs)
  print('Batch size: ', BATCH_SIZE)
  print('Batches per epoch:', batchCount)

  for e in tqdm_notebook(range(1, epoches + 1)):
    if e == 1 or e%plt_frq == 0:
      print('-'*15, 'Epoch %d' %e, '-'*15)
      for _ in range(batchCount):
        # select radom pics in MNIST dataset(real pics)
        image_batch = X_train[np.random.randint(0, X_train.shapw[0], size=BATCH_SIZE)]
        # create random noise
        noise = np.random.normal(0, 1, size=(BATCH_SIZE, z_dim))

        # use G generate pics by noise
        generated_images = g.predict(noise)
        X = np.concatenate((image_batch, generated_images))

        # create label
        y = np.zeros(2*BATCH_SIZE)
        y[:BATCH_SIZE] = 0.9 #

        # train D
        d.trainable = True
        d_loss = d.train_on_batch(X, y)

        # train G
        noise = np.random.normal(0, 1, size=(BATCH_SIZE, z_dim))
        # while train G, assign label equals 1 for pic which were generated by G, trying to trick D
        y2 = np.ones(BATCH_SIZE)
        # train G, stoping updating D paras
        d.trainable = False
        g_loss = gan.train_on_batch(noise, y2)

        # store loss function
      losses['D'].append(d_loss)
      losses['G'].append(g_loss)

        # visualize result
      if e == 1 or e%plt_frq == 0:
        plot_generated()
  plot_loss(losses)


In [0]:
train(epoches=200, plt_frq=20, BATCH_SIZE=128)